
# 🧠 String Interview Playbook — do básico ao avançado

Este notebook reúne **problemas clássicos e alguns avançados de strings**, cada um com **3 versões**:

- **Newbie**: direta, didática — às vezes menos eficiente.
- **Fodona**: solução ótima/algorítmica (janela deslizante, DP, KMP, Manacher, etc.).
- **Pythônica apelona**: usando o poder da linguagem (ex.: `Counter`, `sorted`, `re`) — *quando a entrevista permitir*.

Para cada problema há **explicação**, **complexidade** e **perguntas típicas** de entrevista.


In [62]:

def run_tests(func, cases):
    for i, (args, expected) in enumerate(cases, 1):
        if not isinstance(args, tuple):
            args = (args,)
        got = func(*args)
        assert got == expected, f"case {i} failed: got={got}, expected={expected}, args={args}"
    "✓ all tests passed"


## 1) Reverter string

Reverter uma string significa produzir uma nova sequência com os caracteres na ordem inversa. É uma operação essencial para validar palíndromos, implementar pilhas manuais, fazer parsing de números escritos ao contrário e aparece como aquecimento em entrevistas.
**Exemplo:** se `s = "algoritmo"`, a resposta esperada é `"omtirogla"`.

O detalhe sutil é que, em Python, `str` é imutável: não conseguimos trocar caracteres in-place. Por isso todas as abordagens aqui trabalham sobre uma estrutura auxiliar (lista) ou devolvem uma nova string. Todas custam tempo linear `O(n)`, porque cada caractere precisa ser visitado ao menos uma vez.


### Newbie

Esta versão expõe a mecânica básica de inverter uma string, ideal para explicar dois ponteiros.

**Passos principais**
1. Converter a string imutável em lista (`list(s)`) para permitir trocas.
2. Iniciar dois índices, um no começo (`i`) e outro no fim (`j`).
3. Trocar `a[i]` com `a[j]`, avançar `i += 1` e recuar `j -= 1` repetindo até que `i >= j`.
4. Reconstruir a string com `"".join(a)`.

**Exemplo rápido:** `"abcd"` vira `['a','b','c','d']` → troca `(0,3)` → troca `(1,2)` → `"dcba"`.

**Complexidade:** tempo `O(n)`; espaço `O(n)` para a lista temporária.


In [35]:

def reverse_string_newbie(s: str) -> str:
    a = list(s)
    i, j = 0, len(a) - 1
    while i < j:
        a[i], a[j] = a[j], a[i]
        i += 1; j -= 1
    return "".join(a)

# quick tests
assert reverse_string_newbie("abc") == "cba"


### Fodona

A versão "fodona" assume a mesma estratégia dos dois ponteiros, porém com foco em invariantes e clareza de variáveis, como pediriam em uma entrevista presencial.

**Raciocínio:** enquanto `l < r`, os elementos `a[l]` e `a[r]` estão fora do lugar. Ao trocá-los garantimos que as posições exteriores já ficam corretas. O algoritmo termina quando a janela se fecha, garantindo que todos os pares foram permutados exatamente uma vez.

**Complexidade:** tempo `O(n)`; espaço `O(n)` por causa da conversão da string em lista mutável.


In [36]:

def reverse_string_fodona(s: str) -> str:
    a = list(s)
    l, r = 0, len(a) - 1
    while l < r:
        a[l], a[r] = a[r], a[l]
        l += 1; r -= 1
    return "".join(a)


### Pythônica apelona

Quando a entrevista permite usar recursos de linguagem, `s[::-1]` é a forma mais sucinta e rápida em Python para reverter strings.

**Por que funciona:** o slicing com passo `-1` instrui o interpretador a percorrer a sequência do fim para o começo, alocando uma nova string com os caracteres na ordem inversa.

**Complexidade:** tempo `O(n)` (cada caractere é copiado uma vez); espaço `O(n)` (a nova string).


In [37]:

def reverse_string_pythonica(s: str) -> str:
    return s[::-1]


## 2) Verificar anagramas sem `sorted`

Um anagrama é uma permutação das letras de outra palavra ou frase. "Roma" e "Amor", por exemplo, são anagramas porque usam as mesmas letras com a mesma multiplicidade. Entrevistadores gostam da pergunta porque ela exige raciocinar sobre contagem de frequências e domínio de caracteres, além de evitar a solução fácil `sorted(a) == sorted(b)`.

Analisar anagramas revela diferentes trade-offs: dicionários funcionam para qualquer conjunto de símbolos, enquanto arrays fixos são ultra rápidos quando o alfabeto é conhecido (apenas `a..z`). Todas as versões aqui executam em tempo linear `O(n)`, mas o espaço varia conforme a estrutura auxiliar.


### Newbie

Primeiro construímos um contador de frequências com `dict` para `a` e depois "descontamos" os caracteres de `b`.

**Passos principais**
1. Se tamanhos divergem, não são anagramas.
2. Contar cada letra de `a` usando `cnt[ch] = cnt.get(ch, 0) + 1`.
3. Para cada letra em `b`, decrementar. Se alguma contagem ficar negativa ou inexistente, falha.
4. Se terminamos sem inconsistências, as duas strings usam os mesmos caracteres com a mesma multiplicidade.

**Complexidade:** tempo `O(n)`; espaço `O(k)` onde `k` é a quantidade de símbolos distintos.


In [38]:

def is_anagram_newbie(a: str, b: str) -> bool:
    if len(a) != len(b): return False
    cnt = {}
    for ch in a:
        cnt[ch] = cnt.get(ch, 0) + 1
    for ch in b:
        if cnt.get(ch, 0) == 0:
            return False
        cnt[ch] -= 1
    return True


### Fodona (array 26)

Quando sabemos que o alfabeto é apenas `a..z`, substituímos o `dict` por um vetor de tamanho fixo, o que economiza constante e evita hashing.

**Passos principais**
1. Mapear cada caractere para `idx = ord(ch) - ord('a')`.
2. Incrementar contagens para `a`, decrementar para `b`.
3. Se algum caractere sair do intervalo `0..25`, voltamos para a versão genérica.
4. No final, verificar se todas as contagens zeraram.

**Complexidade:** tempo `O(n)`; espaço `O(1)` (o vetor tem tamanho constante).


In [39]:

def is_anagram_fodona(a: str, b: str) -> bool:
    if len(a) != len(b): return False
    base = ord('a')
    freq = [0]*26
    for ch in a:
        idx = ord(ch)-base
        if 0 <= idx < 26:
            freq[idx]+=1
        else:
            return is_anagram_newbie(a,b)
    for ch in b:
        idx = ord(ch)-base
        if 0 <= idx < 26:
            freq[idx]-=1
            if freq[idx]<0: return False
        else:
            return is_anagram_newbie(a,b)
    return all(v==0 for v in freq)


### Pythônica apelona

`collections.Counter` encapsula a lógica de contagem em C e é ideal quando a entrevista aceita bibliotecas da stdlib.

**Uso:** `Counter(a) == Counter(b)` compara dicionários de contagem, respeitando multiplicidades e ignorando ordem.

**Complexidade:** tempo `O(n)`; espaço `O(k)` para armazenar as frequências.


In [40]:

from collections import Counter
def is_anagram_pythonica(a: str, b: str) -> bool:
    return Counter(a) == Counter(b)


## 3) Primeiro caractere não repetido

O objetivo é encontrar o índice da primeira letra que aparece exatamente uma vez na string. O problema testa se você sabe combinar contagem de frequências com uma segunda passada para encontrar a resposta.

Use casos reais: identificar o primeiro caractere único em identificadores, encontrar o primeiro usuário não duplicado em uma lista, detectar sinais de ruído em streams. O algoritmo clássico faz duas passadas e usa memória proporcional ao número de caracteres distintos.


### Newbie

A estratégia clássica faz duas passadas: contar e depois encontrar o índice do primeiro caractere único.

**Passos principais**
1. Construir `cnt[ch]` com o número de ocorrências de cada caractere.
2. Percorrer novamente `s` e retornar o primeiro índice `i` tal que `cnt[s[i]] == 1`.
3. Se nenhum caractere atender ao critério, devolvemos `-1`.

**Complexidade:** tempo `O(n)`; espaço `O(k)` para o mapa de frequências.


In [41]:

def first_unique_index_newbie(s: str) -> int:
    cnt = {}
    for ch in s:
        cnt[ch] = cnt.get(ch, 0) + 1
    for i, ch in enumerate(s):
        if cnt[ch] == 1:
            return i
    return -1


### Fodona (26)

Restringindo para letras minúsculas, guardamos duas informações em vetores: a posição da primeira ocorrência e o número de ocorrências.

**Raciocínio:** ao percorrer `s`, atualizamos `first[idx]` (se ainda não definido) e incrementamos `count[idx]`. No fim, basta olhar qual índice aparece exatamente uma vez e tem menor posição.

**Complexidade:** tempo `O(n)`; espaço `O(1)` por manter vetores de tamanho constante.


In [42]:

def first_unique_index_fodona(s: str) -> int:
    first = [-1]*26
    count = [0]*26
    base = ord('a')
    for i, ch in enumerate(s):
        idx = ord(ch)-base
        if 0 <= idx < 26:
            if first[idx] == -1: first[idx] = i
            count[idx] += 1
        else:
            return first_unique_index_newbie(s)
    ans = 10**9
    for k in range(26):
        if count[k]==1: ans = min(ans, first[k])
    return -1 if ans==10**9 else ans


### Pythônica apelona

Com `Counter` podemos esconder a contagem de frequências e focar na segunda passada.

**Uso típico:**
```python
cnt = Counter(s)
for i, ch in enumerate(s):
    if cnt[ch] == 1:
        return i
```

**Complexidade:** tempo `O(n)`; espaço `O(k)` para o contador.


In [43]:

from collections import Counter
def first_unique_index_pythonica(s: str) -> int:
    cnt = Counter(s)
    for i,ch in enumerate(s):
        if cnt[ch]==1: return i
    return -1


## 4) Palíndromo válido (ignora não-alfaNum; case-insensitive)

Palíndromos são palavras ou frases que lidas de trás para frente produzem a mesma sequência. Quando ignoramos acentuação, pontuação e caixa, "Socorram-me subi no ônibus em Marrocos" continua sendo palíndromo.

Na entrevista, o desafio está em filtrar apenas caracteres alfanuméricos e normalizar para caixa única antes de comparar. As abordagens variam entre usar dois ponteiros e construir uma nova sequência filtrada.


### Duas pontas

O truque é ignorar tudo o que não for alfanumérico e comparar caracteres já normalizados em caixa baixa.

**Passos principais**
1. Iniciar `i` no começo e `j` no fim.
2. Avançar `i` enquanto `s[i]` não for alfanumérico; o mesmo para `j`.
3. Quando ambos apontarem para letras ou dígitos, comparar `s[i].lower()` com `s[j].lower()`.
4. Repetir até que `i >= j`.

**Complexidade:** tempo `O(n)`; espaço `O(1)` (apenas índices).


In [44]:

def is_palindrome_valid(s: str) -> bool:
    i, j = 0, len(s)-1
    while i < j:
        while i < j and not s[i].isalnum(): i+=1
        while i < j and not s[j].isalnum(): j-=1
        if s[i].lower() != s[j].lower(): return False
        i+=1; j-=1
    return True


### Pythônica apelona

Podemos escrever a mesma ideia de forma declarativa montando uma lista filtrada.

**Passos:**
1. Construir `t = [c.lower() for c in s if c.isalnum()]`.
2. Verificar se `t == t[::-1]`.

**Complexidade:** tempo `O(n)`; espaço `O(n)` devido à lista filtrada.


In [45]:

def is_palindrome_pythonica(s: str) -> bool:
    t = [c.lower() for c in s if c.isalnum()]
    return t == t[::-1]


## 5) Maior substring sem repetir caracteres

Dada uma string, queremos o comprimento da maior substring (contígua) sem caracteres repetidos. Esse problema é o cartão de visitas da técnica de janela deslizante.

Exemplo clássico: para `"abcabcbb"`, a resposta é `3` ("abc"). Esse tipo de lógica aparece em sistemas de stream, buffers de rede e validação de tokens. A dificuldade está em manter o estado da janela sem reprocessar tudo a cada passo.


### Newbie (set + move)

É a versão pedagógica da janela deslizante. Mantemos um `set` com os caracteres atuais e dois ponteiros que delimitam a janela sem repetição.

**Passos principais**
1. Expandir `j` enquanto `s[j]` não estiver no conjunto.
2. Se encontrarmos repetição, remover `s[i]` e avançar `i` até que o conflito desapareça.
3. Atualizar `best` com `j - i`.

**Complexidade:** tempo `O(n)`; espaço `O(k)` para o conjunto (onde `k` é o tamanho da maior janela única).


In [46]:

def length_longest_substring_newbie(s: str) -> int:
    seen = set(); i=j=best=0
    while i<len(s) and j<len(s):
        if s[j] not in seen:
            seen.add(s[j]); j+=1; best=max(best, j-i)
        else:
            seen.remove(s[i]); i+=1
    return best


### Fodona (último índice)

Otimização importante: em vez de remover um a um, guardamos a última posição onde cada caractere apareceu.

**Raciocínio:** quando `ch` reaparece dentro da janela atual (isto é, `last[ch] >= start`), pulamos direto para `last[ch] + 1`. Isso garante que cada índice move-se no máximo uma vez.

**Complexidade:** tempo `O(n)`; espaço `O(k)` para o dicionário de últimas posições.


In [47]:

def length_longest_substring_fodona(s: str) -> int:
    last = {}; start = best = 0
    for i,ch in enumerate(s):
        if ch in last and last[ch] >= start:
            start = last[ch]+1
        last[ch] = i
        if i-start+1 > best: best = i-start+1
    return best


### Pythônica apelona (compacta)

A mesma lógica da versão fodona, mas condensada usando `dict.get` e atualizações inline. Útil para mostrar domínio da linguagem sem perder clareza algorítmica.

**Complexidade:** tempo `O(n)`; espaço `O(k)` (mapa de últimas aparições).


In [48]:

def length_longest_substring_pythonica(s: str) -> int:
    last = {}; start = best = 0
    for i,ch in enumerate(s):
        start = max(start, last.get(ch,-1)+1)
        last[ch] = i
        best = max(best, i-start+1)
    return best


## 6) Substring search (KMP) — sem `find`

Encontrar um padrão `pat` dentro de um texto maior `text` é um dos problemas fundadores de algoritmos de string. A versão ingênua compara o padrão em cada deslocamento possível e custa `O(n·m)` no pior caso.

Knuth–Morris–Pratt (KMP) muda o jogo: ao pré-processar o padrão para saber o maior prefixo que também é sufixo (array `lps`), evitamos recomeçar a busca do zero quando ocorre um mismatch. Assim, ambos os ponteiros avançam monotonicamente e o algoritmo roda em tempo `O(n + m)` com espaço `O(m)` para o `lps`.


### Fodona (KMP)

O KMP começa com o pré-processamento do padrão para gerar o vetor `lps` (longest prefix suffix). Para cada posição `i`, `lps[i]` guarda o tamanho do maior prefixo de `pat` que também é sufixo de `pat[:i+1]`.

**Intuição do `lps`:** ao falhar uma comparação em `pat[i]`, sabemos qual próximo caractere tentar sem voltar o ponteiro do texto. Exemplo com `pat = "ababaca"`: o prefixo `"aba"` coincide com o sufixo que termina em `i = 4`, por isso `lps[4] = 3`.

**Busca:** percorremos o texto com índice `i` e o padrão com `j`. Quando `text[i] != pat[j]`, reduzimos `j = lps[j-1]`. Como `i` nunca recua, visitamos cada caractere de `text` no máximo uma vez.

**Complexidade:** pré-processamento `O(m)` e busca `O(n)`; espaço `O(m)` para o vetor `lps`.


In [49]:

def kmp_search(text: str, pat: str) -> int:
    if pat=="": return 0
    lps = [0]*len(pat)
    i=j=1; j=0
    while i < len(pat):
        if pat[i]==pat[j]:
            j+=1; lps[i]=j; i+=1
        elif j:
            j=lps[j-1]
        else:
            lps[i]=0; i+=1
    i=j=0
    while i < len(text):
        if text[i]==pat[j]:
            i+=1; j+=1
            if j==len(pat): return i-j
        elif j:
            j=lps[j-1]
        else:
            i+=1
    return -1


### Newbie (força bruta)

Antes de aprender KMP, vale revisar o algoritmo ingênuo. Ele testa o padrão em cada deslocamento possível e compara caractere a caractere.

**Complexidade:** tempo `O(n·m)` no pior caso; espaço `O(1)`.


In [50]:

def find_bruteforce(text: str, pat: str) -> int:
    if pat=="": return 0
    n,m=len(text),len(pat)
    for i in range(n-m+1):
        k=0
        while k<m and text[i+k]==pat[k]:
            k+=1
        if k==m: return i
    return -1


### Pythônica apelona

A versão pythonizada delega o trabalho para `text.find(pat)`. Por baixo dos panos o CPython usa uma implementação em C baseada em algoritmos como Two-Way/FASTSEARCH, que é bem mais otimizada que o brute force puro escrito em Python.

**Quando usar:** em entrevistas que aceitam chamadas à biblioteca padrão, reforçando que você entende a teoria (KMP) mas prefere um atalho seguro e testado.

**Complexidade:** no pior caso ainda é `O(n·m)`, porém com constantes muito menores e alguns cenários amortizados quase lineares. Espaço `O(1)`.


In [51]:

def find_pythonica(text: str, pat: str) -> int:
    return text.find(pat)


## 7) Maior substring palindrômica

Aqui buscamos a substring contígua mais longa que seja palíndroma. O problema aparece em compressão, bioinformática e validação de entradas.

Há três famílias de soluções: força bruta (`O(n^3)`), expansão em torno de centros (`O(n^2)`), e Manacher (`O(n)`). Este notebook mostra a expansão de centros como baseline, a implementação completa de Manacher e uma versão pythonizada para quem prioriza legibilidade.


### Newbie (expandir centros)

Cada palíndromo pode ser visto como expandindo a partir de um centro (um caractere para ímpares ou o meio de dois caracteres para pares).

**Passos principais**
1. Para cada índice `i`, expandir em torno de `(i, i)` e `(i, i+1)`.
2. A função `expand` desce à esquerda e sobe à direita enquanto os caracteres coincidirem.
3. Guardar a melhor janela vista.

**Complexidade:** tempo `O(n^2)`; espaço `O(1)`.


In [52]:

def longest_pal_substring_newbie(s: str) -> str:
    if not s: return ""
    def expand(l,r):
        while l>=0 and r<len(s) and s[l]==s[r]:
            l-=1; r+=1
        return l+1, r
    best=(0,1)
    for i in range(len(s)):
        l1,r1=expand(i,i)
        l2,r2=expand(i,i+1)
        if r1-l1>r2-l2 and r1-l1>best[1]-best[0]: best=(l1,r1)
        elif r2-l2>best[1]-best[0]: best=(l2,r2)
    return s[best[0]:best[1]]


### Fodona (Manacher)

Manacher transforma a string em outra com sentinelas (`^`, `#`, `$`) para tratar palíndromos pares e ímpares de forma uniforme. O vetor `p[i]` guarda o raio máximo do palíndromo centrado em `i` na string transformada.

**Intuição:** ao processar `i`, usamos simetria em torno do centro atual `c`. Se já conhecemos um palíndromo que se estende até `r`, podemos copiar o valor espelhado `p[mir]` (onde `mir = 2*c - i`) como ponto de partida. Daí seguimos expandindo apenas quando necessário.

O resultado final é o palíndromo de maior raio encontrado. É o único algoritmo conhecido que resolve o problema em tempo linear `O(n)` com espaço `O(n)`.


In [53]:

def longest_pal_substring_manacher(s: str) -> str:
    if not s: return ""
    t=['^']
    for ch in s: t+=['#',ch]
    t+=['#','$']
    n=len(t); p=[0]*n; c=r=0
    for i in range(1,n-1):
        mir=2*c-i
        if i<r: p[i]=min(r-i,p[mir])
        while t[i+1+p[i]]==t[i-1-p[i]]: p[i]+=1
        if i+p[i]>r: c,r=i,i+p[i]
    max_len=center=max((p[i],i) for i in range(1,n-1))
    start=(center-max_len)//2
    return s[start:start+max_len]


### Pythônica apelona (compacta de centros)

Versão reduzida da expansão de centros, priorizando clareza: iteramos pelos índices, tentamos os dois tipos de centro e atualizamos uma substring candidata.

**Complexidade:** tempo `O(n^2)`; espaço `O(1)`.


In [54]:

def longest_pal_substring_pythonica(s: str) -> str:
    best=""
    for i in range(len(s)):
        for a,b in ((i,i),(i,i+1)):
            l,r=a,b
            while l>=0 and r<len(s) and s[l]==s[r]:
                if r-l+1>len(best): best=s[l:r+1]
                l-=1; r+=1
    return best


## 8) Menor janela contendo `t` em `s`

Dadas duas strings `s` e `t`, queremos a menor janela em `s` que contenha todos os caracteres de `t` (respeitando multiplicidades). É outra aplicação prototípica de janela deslizante com mapa de frequências.

Exemplo: `s = "ADOBECODEBANC"` e `t = "ABC"` produzem "BANC". Essa técnica aparece em buscas por padrões, análise de logs e problemas de DNA.


### Fodona (janela)

A solução ótima usa janela deslizante com mapa de contagem. Mantemos `need[ch]` positivo enquanto ainda falta consumir aquele caractere.

**Dinâmica:** expandimos `j` e diminuímos `missing` quando um caractere ainda é necessário. Assim que `missing == 0`, tentamos contrair `i` para encurtar a janela, atualizando a melhor resposta.

**Complexidade:** tempo `O(n)`; espaço `O(k)` onde `k` é o número de caracteres distintos de `t`.


In [55]:

def min_window(s: str, t: str) -> str:
    if not s or not t: return ""
    need={}
    for ch in t: need[ch]=need.get(ch,0)+1
    missing=len(t); i=start=end=0
    for j,ch in enumerate(s,1):
        if need.get(ch,0)>0: missing-=1
        need[ch]=need.get(ch,0)-1
        while missing==0:
            if end==0 or j-i<end-start: start,end=i,j
            need[s[i]]=need.get(s[i],0)+1
            if need[s[i]]>0: missing+=1
            i+=1
    return s[start:end]


### Newbie (brute force educativo)

Ótima para explicar por que a janela deslizante é necessária. Enumeramos todas as substrings e verificamos se cobrem `t`.

**Complexidade:** tempo `O(n^2·k)` (há `O(n^2)` substrings e cada verificação custa `O(k)`); espaço `O(k)` para o mapa auxiliar.


In [56]:

def min_window_newbie(s: str, t: str) -> str:
    if not s or not t: return ""
    def covers(a: str, t: str) -> bool:
        need={}
        for ch in t: need[ch]=need.get(ch,0)+1
        for ch in a:
            if need.get(ch,0)>0: need[ch]-=1
        return all(v==0 for v in need.values())
    best=""
    for i in range(len(s)):
        for j in range(i+1,len(s)+1):
            sub=s[i:j]
            if (not best or len(sub)<len(best)) and covers(sub,t):
                best=sub
    return best


### Pythônica apelona (Counter)

Mesma lógica da janela otimizando a legibilidade com `Counter`. `need = Counter(t)` inicializa a demanda e o restante do algoritmo espelha a versão fodona.

**Complexidade:** tempo `O(n)`; espaço `O(k)`.


In [57]:

from collections import Counter
def min_window_pythonica(s: str, t: str) -> str:
    need=Counter(t); missing=len(t)
    i=start=end=0
    for j,ch in enumerate(s,1):
        if need[ch]>0: missing-=1
        need[ch]-=1
        while missing==0:
            if end==0 or j-i<end-start: start,end=i,j
            need[s[i]]+=1
            if need[s[i]]>0: missing+=1
            i+=1
    return s[start:end]


## 9) Decodificar `"3[a2[c]]"`

A string de entrada usa uma gramática do tipo `k[substring]`, possivelmente aninhada, significando "repita `substring` `k` vezes". Precisamos reconstruir a string decodificada.

É um ótimo exercício de pilhas: cada `[` empilha o estado atual e cada `]` desempilha, combinando contagem e fragmentos parciais. A solução correta lida com números de vários dígitos, recursão implícita via pilha e concatenação eficiente.


### Fodona (pilhas)

Usamos duas pilhas: uma para contagens (`num_st`) e outra para strings parciais (`str_st`). Sempre que aparece um dígito, acumulamos em `k`. Ao ver `[` empilhamos `k` e o prefixo atual; ao ver `]` repetimos o bloco corrente `rep` vezes e concatenamos com o prefixo anterior.

**Exemplo:** `"3[a2[c]]"` → empilha `3` e `""`, processa `a`, empilha `2` ao encontrar `[`, monta `"cc"`, desempilha e concatena resultando em `"accaccacc"`.

**Complexidade:** tempo `O(n)`; espaço `O(n)` para armazenar estados intermediários em entradas muito aninhadas.


In [58]:

def decode_string(s: str) -> str:
    num_st, str_st = [], []
    cur=[]; k=0
    for ch in s:
        if ch.isdigit():
            k=k*10+(ord(ch)-48)
        elif ch=='[':
            num_st.append(k); str_st.append(cur); k=0; cur=[]
        elif ch==']':
            rep=num_st.pop(); prev=str_st.pop()
            cur = prev + cur*rep
        else:
            cur.append(ch)
    return "".join(cur)


## 10) Distância de edição (Levenshtein)

Levenshtein mede o menor número de operações para transformar uma string em outra usando inserção, deleção e substituição. É pilar em autocorretores, comparação de DNA e análise dif.

A forma iterativa mais conhecida usa programação dinâmica: montamos uma tabela onde `dp[i][j]` representa a distância entre os prefixos `a[:i]` e `b[:j]`. A tabela pode ser comprimida para duas linhas, reduzindo o uso de memória sem perder tempo assintótico (`O(n·m)`).


### Newbie (tabela completa)

Construímos uma matriz `(n+1) x (m+1)` onde cada célula `dp[i][j]` guarda a menor distância entre os prefixos `a[:i]` e `b[:j]`. Os casos base representam converter string vazia em prefixos via inserções ou deleções.

**Recorrência:**
- Inserção: `dp[i][j-1] + 1`
- Remoção: `dp[i-1][j] + 1`
- Substituição ou match: `dp[i-1][j-1] + (a[i-1] != b[j-1])`

O valor final está em `dp[n][m]`.

**Complexidade:** tempo `O(n·m)`; espaço `O(n·m)`.


In [59]:

def edit_distance_newbie(a: str, b: str) -> int:
    n,m=len(a),len(b)
    dp=[[0]*(m+1) for _ in range(n+1)]
    for i in range(n+1): dp[i][0]=i
    for j in range(m+1): dp[0][j]=j
    for i in range(1,n+1):
        for j in range(1,m+1):
            cost=0 if a[i-1]==b[j-1] else 1
            dp[i][j]=min(dp[i-1][j]+1, dp[i][j-1]+1, dp[i-1][j-1]+cost)
    return dp[n][m]


### Fodona (duas linhas)

Observação central: para preencher a linha `i` da tabela só precisamos da linha `i-1` e da linha atual. Assim, armazenamos apenas dois vetores, reaproveitando memória.

**Complexidade:** tempo `O(n·m)`; espaço `O(min(n, m))`, pois sempre iteramos pela string mais curta na dimensão de colunas.


In [60]:

def edit_distance_fodona(a: str, b: str) -> int:
    if len(a) < len(b): a,b=b,a
    prev=list(range(len(b)+1))
    for i,ca in enumerate(a,1):
        cur=[i]
        for j,cb in enumerate(b,1):
            cost=0 if ca==cb else 1
            cur.append(min(prev[j]+1, cur[j-1]+1, prev[j-1]+cost))
        prev=cur
    return prev[-1]


### Pythônica apelona

Não há builtin de Levenshtein no Python padrão. O que podemos fazer durante uma entrevista é negociar o uso de bibliotecas externas (por exemplo, `python-Levenshtein`) ou reaproveitar a versão otimizada de duas linhas mostrada aqui. Reforce o raciocínio: se o avaliador pede algo "idiomático", mostre que já chegamos ao limite teórico com a implementação anterior.


In [61]:

# Não há builtin — a versão 'fodona' já é ideal em Python puro.



---
## Perguntas típicas e como responder

- **Por que sua solução é `O(n)`?**  
  Explico que percorro cada caractere no máximo `k` vezes e mantenho mapas de tamanho limitado.

- **Pode otimizar espaço?**  
  Sim — duas pontas (O(1)) ou DP em duas linhas para Levenshtein.

- **E Unicode?**  
  Trocar array fixo por `dict` e considerar `casefold()`. Para remoção de acentos, normalizar com `unicodedata` (se permitido).

- **Edge cases?**  
  Strings vazias, um caractere, todos iguais, padrão no início/fim, entradas grandes.
